# Execução procedural

## 1. Importando bibliotecas necessárias

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from math import inf

## 2. Estruturação de objetos e funções

### 2.1 Objeto do tipo Cidade (Nome, UF, arestas de conexão)

In [ ]:
class Cidade:
    def __init__(self, Nome, UF):
        self.Nome = Nome
        self.UF = UF
        self.arestas_saida = {}

    def __repr__(self):
        saidas = "\n"
        for cidade, (dist, usa_barco) in self.arestas_saida.items():
            saidas += (
                f"\t→ {cidade.UF:<3} | "
                f"Distancia: {dist:>5} | "
                f"Usa barco: {'Sim' if usa_barco else 'Não'}\n"
            )

        return (
            f"-- {self.Nome} ({self.UF}) --\n"
            f"Arestas de saída:{saidas}\n"
        )

    def __hash__(self):
        return hash(self.UF)

    def __eq__(self, outra):
        return isinstance(outra, Cidade) and self.UF == outra.UF
        

    def adicionar_aresta(self, destino, distancia, usa_barco):
        self.arestas_saida[destino] = (distancia, usa_barco)

### 2.2 Função para carregar dados formato: CIDADE,UF,CONEXAO1:DISTANCIA,CONEXAO2:DISTANCIA,CONEXAO3:DISTANCIA (Uso de * antes do nome CONEXAO identifica se usa barco ou não)

In [ ]:
def carregarCSV(txt):
    with open(txt, 'r', encoding='utf-8') as arq:
        dados = arq.readlines()
    
    cidades = {}

    #Criar cidades
    for linha in dados:
        linha = linha.strip()
        aux = linha.split(",")

        nome = aux[0]
        uf = aux[1]

        cidades[uf] = Cidade(nome, uf)

    #Criar vizinhanças
    for linha in dados:
            linha = linha.strip()
            aux = linha.split(",")
    
            cidade_atual = cidades[aux[1]]
    
            for coluna in aux[2:]:
                usa_barco = False
    
                if coluna[0] == "*":
                    coluna = coluna[1:]
                    usa_barco = True
    
                destino_uf, dist = coluna.split(":")
                dist = int(dist)
    
                cidade_atual.adicionar_aresta(
                    cidades[destino_uf],
                    dist,
                    usa_barco
                )     
    return list(cidades.values())

### 2.3 Dijkstra que calcula a menor distancia entre a cidade atual e as cidades adjacentes

In [ ]:
def dijkstra(cidades, origem:str, recebe_penalidade:bool = False, multiplicador_penalidade:float = 1.5):
    """
    Calcula a menor distancia 
    dijkstra(cidades, origem, recebe_penalidade, multiplicador_penalidade)

    cidades -> Lista de objetos Cidade
    origem -> String referente ao UF da cidade. Ex.: "RJ", "SP", "RS", "PE"
    recebe_penalidade -> Booleano que indica se barcos aumentam ou não o peso da distancia
    multiplicador_penalidade -> Indica a penalidade que barcos aplicam no peso sendo 1> incremental, e <1 decremental
    """
    # Dicinários de objetos cidade com os valores iniciais
    distancia = {cidade: inf for cidade in cidades}
    pai = {cidade: None for cidade in cidades}
    visitado = {cidade: False for cidade in cidades}

    distancia[origem] = 0

    while True:
        # Escolher a cidade não visitada com menor distância
        u = None
        menor_dist = inf

        for cidade in cidades:
            if not visitado[cidade] and distancia[cidade] < menor_dist:
                menor_dist = distancia[cidade]
                u = cidade

        # Se não achou nenhuma, terminou
        if u is None:
            break

        visitado[u] = True
        for v, (peso, usa_barco) in u.arestas_saida.items():
            if not visitado[v]:
                peso = peso*multiplicador_penalidade if usa_barco and recebe_penalidade else peso
                nova_dist = distancia[u] + peso
                if nova_dist < distancia[v]:
                    distancia[v] = nova_dist
                    pai[v] = u

    return distancia, pai

### 2.4 Função que auxilia o algoritmo de dijkstra a construir o caminho/rota até a cidade destino

In [ ]:
def construir_rota(pai, origem, destino):
    caminho = []
    atual = destino

    while atual is not None:
        caminho.append(atual)
        atual = pai[atual]

    caminho.reverse()

    if caminho[0] != origem:
        return None

    return caminho

### 2.5 Funções de auxílio gráfico

#### 2.5.1 Carrega mapa base do Brasil

In [ ]:
def carregar_mapa_brasil():
    return gpd.read_file(
        "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
    )

#### 2.5.2 Calcula o centroide do estado baseado no UF informado

In [ ]:
def centroides_por_uf(mapa):
    coords = {}

    for _, row in mapa.iterrows():
        uf = row["sigla"]
        centro = row.geometry.centroid
        coords[uf] = (centro.x, centro.y)

    return coords

#### 2.5.3 Desenha mapa do Brasil obtido no Geopandas

In [ ]:
def desenhar_mapa_base(mapa):
    fig, ax = plt.subplots(figsize=(12, 8), dpi=100)
    plt.subplots_adjust(right=0.75)

    mapa.plot(
        ax=ax,
        color="whitesmoke",
        edgecolor="black",
        linewidth=0.8
    )

    ax.set_title("Mapa do Brasil - Estados")
    ax.axis("off")

    return fig, ax

#### 2.5.4 Plota as linhas de conexão entre os dados, com uma tabela de legenda informando a distancia parcial, total e meio de transporte. (Leva em consideração se houve penalidade ou não)

In [ ]:
def desenhar_caminho(ax, caminho, coords, recebe_penalidade:bool=False, multiplicador_penalidade:float = 1.5):
    distancia_total = 0
    linhas_legenda = []

    for cidade in caminho:
        x, y = coords[cidade.UF]
        ax.plot(x, y, "o", color="blue", markersize=2)
        ax.text(x, y, cidade.UF, fontsize=15, ha="center", va="center")

    for i in range(len(caminho) - 1):
        c1 = caminho[i]
        c2 = caminho[i + 1]

        x1, y1 = coords[c1.UF]
        x2, y2 = coords[c2.UF]

        dist, usa_barco = c1.arestas_saida[c2]

        # Penalidade
        penalidade = (multiplicador_penalidade-1) * dist if usa_barco and recebe_penalidade else 0
        dist_exibida = dist + penalidade
        distancia_total += dist_exibida

        # Cor e tipo
        cor = "blue" if usa_barco else "black"
        tipo = "Barco" if usa_barco else "Terra"

        # Desenhar seta
        ax.annotate(
            "",
            xy=(x2, y2),
            xytext=(x1, y1),
            arrowprops=dict(
                arrowstyle="->",
                color=cor,
                linewidth=1.5
            )
        )

        xm = (x1 + x2) / 2
        ym = (y1 + y2) / 2

        texto_mapa = (
            f"{dist} + {int(penalidade)}"
            if usa_barco and recebe_penalidade else f"{dist}"
        )

        ax.text(
            xm,
            ym,
            texto_mapa,
            fontsize=10,
            color=cor,
            ha="center",
            va="center",
            bbox=dict(
                boxstyle="round,pad=0.2",
                fc="white",
                ec=cor,
                alpha=0.85
            )
        )

        linhas_legenda.append(
            f"{c1.UF} → {c2.UF} | {tipo} | {dist} + {int(penalidade)} = {int(dist_exibida)}"
            if usa_barco and recebe_penalidade
            else f"{c1.UF} → {c2.UF} | {tipo} | {dist}"
        )

    legenda_texto = "Rota por segmento\n\n"
    legenda_texto += "\n".join(linhas_legenda)
    legenda_texto += f"\n\nDistância total: {int(distancia_total)}"

    """ax.text(
        1.02, 0.5,
        legenda_texto,
        transform=ax.transAxes,
        fontsize=10,
        va="center",
        ha="left",
        bbox=dict(
            boxstyle="round,pad=0.5",
            fc="white",
            ec="black",
            alpha=0.9
        )
    )"""
    
    titulo = "Com penalidades" if recebe_penalidade else "Sem penalidades"

    ax.text(
        1.02, 0.95,
        titulo,
        transform=ax.transAxes,
        fontsize=12,
        fontweight="bold",
        va="top",
        ha="left"
    )

    ax.plot([1.02, 1.25], [0.93, 0.93], transform=ax.transAxes, color="black", lw=1)
    
    ax.text(
        1.02, 0.90,
        legenda_texto,
        transform=ax.transAxes,
        fontsize=10,
        va="top",
        ha="left"
    )

## 3. Execução do dijkstra

### 3.1 Caminho até os dados

In [ ]:
dados = carregarCSV("../data/processed/cidades.txt")

### 3.2 Definição de UF Origem e UF Destino

In [ ]:
UF_Origem = 'RJ'
UF_destino = 'AM'

### 3.3 Parâmetros de penalidade (se aplicável Barcos serão X vezes mais impactantes no peso da distância)

In [ ]:
usaPenalidade = True

pesoPenalidade = 1.4

### 3.4 Execução e exibição dos resultados

In [ ]:
origem = next(c for c in dados if c.UF == UF_Origem)
destino = next(c for c in dados if c.UF == UF_destino)

dist, pai = dijkstra(dados, origem, recebe_penalidade = usaPenalidade, multiplicador_penalidade = pesoPenalidade)
caminho = construir_rota(pai, origem, destino)

distancia_anterior = dist[origem]

print(f"\nDijkstra - {origem.UF} → {destino.UF}\n")
print(f"{'Cidade':<25} {'UF':<4} {'Trecho':>12} {'Acumulado':>14}")
print("-" * 60)

for c in caminho:
    trecho = dist[c] - distancia_anterior
    acumulado = dist[c]

    print(
        f"{c.Nome:<25} "
        f"{c.UF:<4} "
        f"{trecho:>12.1f} "
        f"{acumulado:>14.1f}"
    )

    distancia_anterior = dist[c]

## 4. Plotagem gráfica

In [ ]:
mapa = carregar_mapa_brasil()
coords = centroides_por_uf(mapa)
fig, ax = desenhar_mapa_base(mapa)
desenhar_caminho(ax, caminho, coords, recebe_penalidade = usaPenalidade, multiplicador_penalidade = pesoPenalidade)

# Execução em GPU